# Self-contained Robo-advisor

In [ ]:
import pandas as pd
import plotly.graph_objects as go

from PortfolioPredictor import evaluate_portfolios_over_time, extract_asset_returns, chain_portfolio_performance
from fetchData import fetch_raw_data_yf, getSNP500, getNasdaq_comp, fetch_raw_data_yf_all
from LearningRBA import MLRBA_V2
from PortfolioFunction import get_matrices

In [ ]:
assets = getSNP500()
start_date = pd.Timestamp("2014-01-01")
end_date = pd.Timestamp("2016-01-01")
investment_period = 910
investment_end_date = end_date + pd.Timedelta(days=investment_period)
raw_data, _, _= fetch_raw_data_yf(assets, start_date, investment_end_date)

raw_data_train = raw_data.loc[:end_date]
raw_data_test = raw_data.loc[end_date:]

In [ ]:
names, annualized_returns, x, x1, _, cov, correlation_matrix = get_matrices(raw_data[:end_date])
_, initial_portfolio, _, _, _ = MLRBA_V2(names, cov, annualized_returns, correlation_matrix)

In [ ]:
from PortfolioPredictor2 import PortfolioPredictorDirectMultiStep

portfolio_predictor = PortfolioPredictorDirectMultiStep(raw_data_train, raw_data_test, initial_portfolio, n_steps=5)

portfolio_predictor.preprocess_data()
portfolio_predictor.build_model()
portfolio_predictor.train_model()
prediction = portfolio_predictor.predict()   
portfolio_predictor.plot_loss()
portfolio_predictor.plot_predictions()
portfolio_predictor.compute_performance()

In [ ]:
import numpy as np

data = np.array([[i] for i in range(30)])  # Convert to 2D array with shape (30, 1)

def create_datasets(lookback, n_steps, data):
    X, y = [], []
    for i in range(len(data) - lookback - n_steps + 1):
        X.append(data[i: i + lookback, :])
        target = data[i + lookback: i + lookback + n_steps, 0]
        y.append(target)
    return np.array(X), np.array(y)

x, y = create_datasets(10, 5, data)